# Đồ án cuối kỳ: Dự báo thời tiết
<br>
NGƯỜI THỰC HIỆN: TRẦN XUÂN LỘC - 18127131<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;THÁI HOÀNG HUY - 18127109<br>
LỚP: 18KHMT<br>
Giáo viên hướng dẫn: TRẦN TRUNG KIÊN



# Import

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
from calendar import monthrange
import json
import re
import requests_cache
import pandas as pd

In [2]:
requests_cache.install_cache(expire_after=-1)

# Thu thập dữ liệu
- Dữ liệu sử dụng trong đồ án lần này được lấy từ api của trang web [Ambee Weather](https://www.amberweather.com/); Trong quá trình thu thập dữ liệu từ API do số lượng mẫu thu thập rất lớn khoảng 9000 mẫu mà API được cung cấp chỉ giới hạn 500 mẫu 1 ngày nên việc thu thập dữ liệu rất tốn thời gian và dữ liệu tại một số địa điểm có thể không được cung cấp đầy đủ bởi trang web đó.

# Định nghĩa các hàm thu thập dữ liệu

- Hàm **get_data** nhận thông tin tọa độ kinh độ (longitude) dưới dạng biến *lng* và vĩ độ (latitude) dưới dạng biên *lat* và key thể hiện cho khóa API mà trang web cung cấp cho tài khoản của ta trong quá trình đăng kí tài khoản.
- Hàm thu thập dữ liệu 13 ngày đầu tiên trong mỗi tháng vào 3 khoảng thời gian trong ngày từ 7-9h sáng, 11-13h chiều và 16-18h tối từ tháng 1 tới tháng 12 năm 2020. Khoảng thời gian cách nhau mỗi 2 tiếng vì để đảm bảo cho API trả về ít nhất là 1 kết quả do có địa điểm có dữ liệu theo từng tiếng và có những địa điểm khi cho khoảng thời gian cách nhau 2 tiếng thì mới tồn tại ít nhất 1 mẫu. Ngoài ra chia ra 3 thời điểm thu thập dữ liệu để đảm bảo có đủ dữ liệu để đánh giá thời tiết trong 3 ngày đó.
- Trong hàm getdata gọi hàm request_data để lấy dữ liệu từ API tại khoảng thời gian và kinh độ, vĩ độ của quốc gia cần lấy, sau đó thêm vào mảng *list_request* và mảng *list_request* này chứa dữ liệu dưới dạng thô chưa xử lí.

In [3]:
def get_data(lat,lng,key):
    date01 = datetime.strptime("2020-01-01 07:00:00", '%Y-%m-%d %H:%M:%S')
    date02 = datetime.strptime("2020-01-01 09:00:00", '%Y-%m-%d %H:%M:%S')
    date03 = datetime.strptime("2020-01-01 11:00:00", '%Y-%m-%d %H:%M:%S')
    date04 = datetime.strptime("2020-01-01 13:00:00", '%Y-%m-%d %H:%M:%S')
    date05 = datetime.strptime("2020-01-01 16:00:00", '%Y-%m-%d %H:%M:%S')
    date06 = datetime.strptime("2020-01-01 18:00:00", '%Y-%m-%d %H:%M:%S')
    list_request = []
    error = False
    for i in range(12):
        tmp01 = date01
        tmp02 = date02
        tmp03 = date03
        tmp04 = date04
        tmp05 = date05
        tmp06 = date06

        for j in range(13):
            list_request.append(request_data(lat,lng,tmp01.strftime('%Y-%m-%d %H:%M:%S'), tmp02.strftime('%Y-%m-%d %H:%M:%S'),key))
            list_request.append(request_data(lat,lng,tmp03.strftime('%Y-%m-%d %H:%M:%S'), tmp04.strftime('%Y-%m-%d %H:%M:%S'),key))
            list_request.append(request_data(lat,lng,tmp05.strftime('%Y-%m-%d %H:%M:%S'), tmp06.strftime('%Y-%m-%d %H:%M:%S'),key))
            tmp01 = tmp01 + timedelta(days = 1)
            tmp02 = tmp02 + timedelta(days = 1)
            tmp03 = tmp03 + timedelta(days = 1)
            tmp04 = tmp04 + timedelta(days = 1)
            tmp05 = tmp05 + timedelta(days = 1)
            tmp06 = tmp06 + timedelta(days = 1)
            if list_request[-1] == 'error':
                error = True
                break
        if error:
            break
        days = monthrange(date01.year, date01.month)[1] # num_days = 28
        date01 = date01 + timedelta(days = days)
        date02 = date02 + timedelta(days = days)
        date03 = date03 + timedelta(days = days)
        date04 = date04 + timedelta(days = days)
        date05 = date05 + timedelta(days = days)
        date06 = date06 + timedelta(days = days)
    return list_request, error

- Hàm **request_data** nhận thông tin kinh độ, vĩ độ và thời điểm bắt đầu, thời điểm kết thúc dưới dạng biến start và end, biến key đã giải thích ở trên.
- Hàm thực hiện lấy dữ liệu dưới dạng truy vấn *querystring* nếu phản hồi trả về dưới dạng *response* nếu *status_code* trả về có giá trị 200 có nghĩa là kết quả tồn tại. Nếu rơi vào các trường hợp khác như hết hạn hoặc vượt mức hằng ngày thì biến *status_code* có giá trị khác như 401, 400,...

In [4]:
def request_data(lat,lng,start,end,key):
    url = "https://api.ambeedata.com/weather/history/by-lat-lng"
    querystring = {"lat":lat,"lng":lng,"from":start,"to":end}
    headers = {
    'x-api-key': key,
    'Content-type': "application/json"
    }
    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if response.status_code != 200 :
        return "error"
    
    return response.text

- Hàm **Converter** nhận biến *pattern* chính là các mẫu dữ liệu thu thập từ API mà chúng ta đã lưu trong mảng *list_request* sau đó thực hiện parse dữ liệu lưu vào biến *data* dưới dạng dictionary, hàm xét nếu *pattern[i]* có dạng là dictionary có nghĩa là có khóa mới trong *pattern[i]* hoặc nếu tồn tại 2 mẫu được thu thập trong cùng 1 thời điểm đó thì ta thực hiện lấy phần tử đầu tiên ở dòng *elif* đầu tiên và hàm được khởi tạo dưới dạng đệ qui để khám phá hết cấu trúc bên trong của 1 mẫu dữ liệu thô thu thập từ API.

In [5]:
def Converter(pattern):
    data = {}
    for i in pattern.keys():
        if isinstance(pattern[i], dict):
            child = Converter(pattern[i])
            for j in child.keys():
                data[j] = child[j]
        elif isinstance(pattern[i], list):
            child = Converter(pattern[i][0])
            for j in child.keys():
                data[j] = child[j]
        else:
            data[i] = pattern[i]
            
    return data

- Hàm **save_data** nhận các biến data, filename, flag lần lượt là dữ liệu có cấu trúc được lưu dưới dạng *list* chứa các thông tin về thời tiết, biến *filename* là tên tệp được lưu trữ có cấu trúc là "tên quốc gia" + ".csv" và biến *flag* có giá trị True/False để viết dòng đầu tiên vào file.

In [6]:
def save_data(data, filename, flag):
    saved_data = folder_path + filename+'.csv'
    collected_data = [json.loads(i) for i in data]
    with open(saved_data, 'a') as f:
        for pattern in collected_data:
            tmp = Converter(pattern)
            if not flag:
                f.write(",".join(tmp.keys()) + ",Country" + '\n')
                flag = True
            f.write(",".join([str(e) for e in tmp.values()]) + ","+ filename+ '\n')
        f.close()

- Biến *folder_path* dùng để chỉ đường dẫn lưu trữ dữ liệu thu thập được dưới dạng ".csv", biến *key* là khóa API được [Ambee website](https://api-dashboard.getambee.com/#/) cung cấp. Biến *positions* và *countries* chứa tọa độ và tên quốc gia mà chúng ta định thu thập dữ liệu, các quốc gia này được chọn ngẫu nhiên và có vị trí địa lý và kiểu khí hậu khác nhau trên toàn thế giới.

In [7]:
folder_path = 'data//'
key = "b76f1aa3c1970f5a6318ad1aafdc89676256149755894c754454eb0a7e8f3955"
positions =[["56.130366", "-106.346771"],["60.128161", "18.643501"], ["35.907757", "127.766922"], ["-9.189967", "-75.015152"], ["71.706936", "-42.604303"], ["-38.416097", "-63.616672"], ["9.145", "40.489673"], ["23.634501", "-102.552784"], ["26.820553", "30.802498"], ["-0.789275", "113.921327"], ["36.204824", "138.252924"], ["35.86166", "104.195397"],["47.516231", "14.550072"],["55.378051", "-3.435973"], ["9.081999", "8.675277"],["10.82302", "106.62965"],["-30.559482", "22.937506"],["37.09024", "-95.712891"],["51.919438", "19.145136"],["61.52401", "105.318756"], ["53.41291","-8.24389"]]
countries = ["Canada","Sweden", "South Korea", "Peru", "Greenland", "Argentina", "Ethiopia", "Mexico", "Egypt", "Indonesia", "Japan", "China", "Austria","United Kingdom","Nigeria","VietNam","South Africa","United States", "Poland","Russia","Ireland"]

- Tiếp theo, chúng ta sẽ kiểm tra xem có bao nhiêu quốc gia.

In [8]:
print(len(positions), len(countries))

21 21


## Thu thập dữ liệu từ 1 quốc gia (có thể chạy demo tính năng thu thập dữ liệu)

- Do dữ liệu thu thập một ngày bị giới hạn 500 mẫu, do đó nếu cần thử nghiệm tính năng thu thập dữ liệu ta sẽ chạy cell bên dưới, để thu thập dữ liệu từ nước **Sweden**, ta gọi hàm **get_data** nhập kinh độ, vĩ độ của nước đó là lưu dữ liệu vào biến *data* sau đó gọi hàm **save_data** truyền biến *data* là dữ liệu thu thập được bên trên và tên nước *country* ở đây là nước **Sweden**.

In [ ]:
def Demo(country, lat, lng):
    data, status = get_data(lat, lng, key)
    if status:
        print("Bad API")
        return
    else:
        save_data(data, country, False)
        print("Collected data from ", country, "sucessfully !")

Demo(country="Sweden", lat="60.128161", lng="18.643501")

## Thu thập dữ liệu từ 21 quốc gia (Not Run).

- Hàm *request_data_21_country* thực hiện lấy dữ liệu của 21 quốc gia.

In [10]:
def request_data_21_country(ket, positions,countries):
    for pos, country in zip( positions,countries):
        flag = False
        data = get_data(pos[0],pos[1], key)
        save_data(data, country, flag)
        print("Collected data from ", country, " sucessfully !")
        time.sleep(1)